# Week 10: Information Retrieval and Question Answering

This week we are looking at the tasks of information retrieval and question answering.  In the lab we will look at:
* finding relevant documents to a query expressed in terms of keywords
* extracting keywords from a query

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive/')

In [1]:
#preliminary imports
import pandas as pd
from itertools import zip_longest
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import math
import operator

import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kerimciger/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kerimciger/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

We will be using the Stanford Question Answering Dataset (SQUAD) dataset in this lab, provided in the resources for this week.  For more information about the dataset and the ongoing competition, see the website:  https://rajpurkar.github.io/SQuAD-explorer/explore/1.1/dev/

The dataset is provided as 2 json files - one for development and one for training.  Lets read in the smaller development dataset using the json library.

In [7]:
import json
import os
#filename="/content/drive/My Drive/NLE Notebooks/Week10LabsSolutions/Squad/dev-v2.0.json"
#path="/Users/home/juliewe/Documents/teaching/NLE/NLE2021/w11/Week11LabsSolutions/Week11Labs"
filename=f'{os.getcwd()}/Squad/dev-v2.0.json'
with open(filename,'r') as inputstream:
    devdata=json.load(inputstream)
    


The data is now stored in devdata as a deeply-nested dictionary structure.  The `data` field contains a list of dictionaries - one for each of the documents in the dataset.

In [8]:
len(devdata['data'])

35

Each document dictionary has two key-value pairs.

In [9]:
len(devdata['data'][0].keys())

2

In [12]:
list(devdata['data'][0].keys())

['title', 'paragraphs']

The `paragraphs` field of a document dictionary is a further list of paragraphs!

In [13]:
len(devdata['data'][0]['paragraphs'])

39

Each paragraph is a dictionary.  One field `context` contains the text of the paragraph as a String.  

In [14]:
list(devdata['data'][0]['paragraphs'][0].keys())

['qas', 'context']

In [15]:
devdata['data'][0]['paragraphs'][0]['context']

'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.'

The other field `qas` is a list of dictionaries.  Each dictionary contains a `question`, a list of possible `answers`, a unique `id` and a value for `is_impossible` which indicates whether the question is answerable or not given the `context`

In [16]:
devdata['data'][0]['paragraphs'][0]['qas']

[{'question': 'In what country is Normandy located?',
  'id': '56ddde6b9a695914005b9628',
  'answers': [{'text': 'France', 'answer_start': 159},
   {'text': 'France', 'answer_start': 159},
   {'text': 'France', 'answer_start': 159},
   {'text': 'France', 'answer_start': 159}],
  'is_impossible': False},
 {'question': 'When were the Normans in Normandy?',
  'id': '56ddde6b9a695914005b9629',
  'answers': [{'text': '10th and 11th centuries', 'answer_start': 94},
   {'text': 'in the 10th and 11th centuries', 'answer_start': 87},
   {'text': '10th and 11th centuries', 'answer_start': 94},
   {'text': '10th and 11th centuries', 'answer_start': 94}],
  'is_impossible': False},
 {'question': 'From which countries did the Norse originate?',
  'id': '56ddde6b9a695914005b962a',
  'answers': [{'text': 'Denmark, Iceland and Norway', 'answer_start': 256},
   {'text': 'Denmark, Iceland and Norway', 'answer_start': 256},
   {'text': 'Denmark, Iceland and Norway', 'answer_start': 256},
   {'text': 'Den

To make it easier to work with, lets turn this data structure into 2 simple lists containing the bits that we want.  The first will contain the `context` from the paragraphs and the other will contain the corresponding lists of questions and answers or `qas`.

In [17]:
def get_text_qas(dataset):
    textlist=[]
    qalist=[]
    for document in dataset['data']:
        for para in document['paragraphs']:
            textlist.append(para['context'])
            qalist.append(para['qas'])
    
    
    return textlist,qalist
    
    
textlist,qalist=get_text_qas(devdata)

In [18]:
print(textlist[1:2])
print(qalist[1:2])

['The Norman dynasty had a major political, cultural and military impact on medieval Europe and even the Near East. The Normans were famed for their martial spirit and eventually for their Christian piety, becoming exponents of the Catholic orthodoxy into which they assimilated. They adopted the Gallo-Romance language of the Frankish land they settled, their dialect becoming known as Norman, Normaund or Norman French, an important literary language. The Duchy of Normandy, which they formed by treaty with the French crown, was a great fief of medieval France, and under Richard I of Normandy was forged into a cohesive and formidable principality in feudal tenure. The Normans are noted both for their culture, such as their unique Romanesque architecture and musical traditions, and for their significant military accomplishments and innovations. Norman adventurers founded the Kingdom of Sicily under Roger II after conquering southern Italy on the Saracens and Byzantines, and an expedition o

## Keyword Search
In order to perform a keyword search of a document collection, the collection needs to be indexed by words.  The index is a mapping from keywords to lists of document ids containing that document.

Here, we will consider each paragraph as an individual document and its **id** is its position in the `textlist` list created above

### Exercise 1.1
Write a function `extract_keywords()` which takes a string and returns a set of keywords.  Make sure you carry out appropriate normalisation and filtering

In [20]:
def normalise(tokenlist):
    tokenlist=[token.lower() for token in tokenlist]
    tokenlist=["NUM" if token.isdigit() else token for token in tokenlist]
    tokenlist=["Nth" if (token.endswith(("nd","st","th")) and token[:-2].isdigit()) else token for token in tokenlist]
    tokenlist=["NUM" if re.search(r"^[+-]?[0-9]+\.[0-9]",token) else token for token in tokenlist]
    return tokenlist

def filter_stopwords(tokenlist):
    stop = stopwords.words('english')
    return [w for w in tokenlist if w.isalpha() and w not in stop]

def stem(tokenlist):
    st=PorterStemmer()
    return [st.stem(token) for token in tokenlist]

def make_bow(somestring):
    rep=word_tokenize(somestring)  #step 1
    rep=normalise(rep)   #step 2
    rep=stem(rep)   #step 3
    rep=filter_stopwords(rep)  #step 4
    dict_rep={}
    for token in rep:
        dict_rep[token]=dict_rep.get(token,0)+1  #step 5
    return(dict_rep)

In [21]:
def extract_keywords(somestring):
    return set(make_bow(somestring).keys())

In [22]:
extract_keywords(textlist[1])

{'accomplish',
 'adopt',
 'adventur',
 'africa',
 'antioch',
 'architectur',
 'assimil',
 'battl',
 'becom',
 'behalf',
 'bohemond',
 'britain',
 'byzantin',
 'canari',
 'cathol',
 'centr',
 'christian',
 'coast',
 'cohes',
 'conquer',
 'conqueror',
 'conquest',
 'crown',
 'crusad',
 'cultur',
 'dialect',
 'duchi',
 'duke',
 'dynasti',
 'east',
 'england',
 'europ',
 'european',
 'even',
 'eventu',
 'expedit',
 'expon',
 'fame',
 'feudal',
 'fief',
 'forg',
 'form',
 'formid',
 'found',
 'franc',
 'frankish',
 'french',
 'great',
 'hast',
 'ii',
 'impact',
 'import',
 'influenc',
 'innov',
 'ireland',
 'island',
 'itali',
 'kingdom',
 'known',
 'land',
 'languag',
 'led',
 'levant',
 'literari',
 'major',
 'martial',
 'mediev',
 'militari',
 'music',
 'near',
 'new',
 'norman',
 'normandi',
 'normaund',
 'north',
 'note',
 'num',
 'orthodoxi',
 'pieti',
 'polit',
 'princ',
 'princip',
 'richard',
 'roger',
 'romanesqu',
 'saracen',
 'scotland',
 'settl',
 'sicili',
 'signific',
 'south

### Exercise 1.2
Create a keyword index for the paragraphs in textlist.  

In [23]:
def create_index(stringlist):
    index={}
    for i,para in enumerate(stringlist):
        words=extract_keywords(para)
        for word in words:
            current=index.get(word,[])
            current.append(i)
            index[word]=current
    return index

ind=create_index(textlist)
            
        

In [24]:
ind.get('hast',[])

[1, 16]

### Exercise 1.3
Write a class `docssearch` that is initialised with a list of strings, stores the list of strings and also creates an index.  Then write a method to search the keyword index for a single keyword and returns a list of paragraphs containing that keyword.  



In [25]:
class docssearch():
    
    def __init__(self,docs):
        self.docs=docs
        self.index=create_index(docs)        

    def search(self,keyword):
        return [self.docs[index] for index in self.index.get(keyword,[])]
        
    

In [26]:
paras=docssearch(textlist)
paras.search('norman')[:5]

['The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 'The Norman dynasty had a major political, cultural and military impact on medieval Europe and even the Near East. The Normans were famed for their martial spirit and eventually for their Christian piety, becoming exponents of the Catholic orthodoxy in

In [27]:
paras.search('duke')

['The Norman dynasty had a major political, cultural and military impact on medieval Europe and even the Near East. The Normans were famed for their martial spirit and eventually for their Christian piety, becoming exponents of the Catholic orthodoxy into which they assimilated. They adopted the Gallo-Romance language of the Frankish land they settled, their dialect becoming known as Norman, Normaund or Norman French, an important literary language. The Duchy of Normandy, which they formed by treaty with the French crown, was a great fief of medieval France, and under Richard I of Normandy was forged into a cohesive and formidable principality in feudal tenure. The Normans are noted both for their culture, such as their unique Romanesque architecture and musical traditions, and for their significant military accomplishments and innovations. Norman adventurers founded the Kingdom of Sicily under Roger II after conquering southern Italy on the Saracens and Byzantines, and an expedition o

In [28]:
paras.search("hast")

['The Norman dynasty had a major political, cultural and military impact on medieval Europe and even the Near East. The Normans were famed for their martial spirit and eventually for their Christian piety, becoming exponents of the Catholic orthodoxy into which they assimilated. They adopted the Gallo-Romance language of the Frankish land they settled, their dialect becoming known as Norman, Normaund or Norman French, an important literary language. The Duchy of Normandy, which they formed by treaty with the French crown, was a great fief of medieval France, and under Richard I of Normandy was forged into a cohesive and formidable principality in feudal tenure. The Normans are noted both for their culture, such as their unique Romanesque architecture and musical traditions, and for their significant military accomplishments and innovations. Norman adventurers founded the Kingdom of Sicily under Roger II after conquering southern Italy on the Saracens and Byzantines, and an expedition o

The `merge` function below can be used to efficiently intersect two sorted lists.  It is based on the *merge* of *mergesort*.   You might want to test it out on some other lists.  Remember it will only work if the inputs are sorted.

In [31]:
def merge(list1,list2):
    '''
    function to intersect 2 sorted lists
    '''
    pointer1=0
    pointer2=0
    merged=[]
    while pointer1<len(list1) and pointer2<len(list2):
        #print( f"Pointers: {pointer1} {pointer2}")
        if list1[pointer1]==list2[pointer2]:
            merged.append(list1[pointer1])
            pointer1+=1
            pointer2+=1
            
        elif list1[pointer1]<list2[pointer2]:
            pointer1+=1
        else:
            pointer2+=1
            
    return merged

merge([1,5,7,8],[3,6,7,8,15])
        

[7, 8]

### Exercise 1.4
Add a method `listsearch` that takes a list of keywords and returns a list of paragraphs which contains all of them.

Make sure your list of keywords is normalised and filtered in the same way as your paragraph text.

In [32]:
class docssearch():
    
    def __init__(self,docs):
        self.docs=docs
        self.index=create_index(docs)        

    def search(self,keyword):
        return [self.docs[index] for index in self.index.get(keyword,[])]
        
    def listsearch(self,keywordlist):
        keywordlist=stem(filter_stopwords(normalise(keywordlist)))
        if len(keywordlist)>0:
            selected =self.index.get(keywordlist[0],[])
            for keyword in keywordlist[1:]:
                selected=merge(selected,self.index.get(keyword,[]))
        return [self.docs[index] for index in selected]
        

In [33]:
paras=docssearch(textlist)
paras.listsearch(['norman','duke'])

['The Norman dynasty had a major political, cultural and military impact on medieval Europe and even the Near East. The Normans were famed for their martial spirit and eventually for their Christian piety, becoming exponents of the Catholic orthodoxy into which they assimilated. They adopted the Gallo-Romance language of the Frankish land they settled, their dialect becoming known as Norman, Normaund or Norman French, an important literary language. The Duchy of Normandy, which they formed by treaty with the French crown, was a great fief of medieval France, and under Richard I of Normandy was forged into a cohesive and formidable principality in feudal tenure. The Normans are noted both for their culture, such as their unique Romanesque architecture and musical traditions, and for their significant military accomplishments and innovations. Norman adventurers founded the Kingdom of Sicily under Roger II after conquering southern Italy on the Saracens and Byzantines, and an expedition o

A search for paragraphs relevant to the keywords 'political' and 'kingdom' should return 2 paragraphs.  The first starts 'The Norman dynasty ...' and the second starts 'Throughout the 1980s and 1990s ...'

In [34]:
paras.listsearch(['kingdom','political'])

['The Norman dynasty had a major political, cultural and military impact on medieval Europe and even the Near East. The Normans were famed for their martial spirit and eventually for their Christian piety, becoming exponents of the Catholic orthodoxy into which they assimilated. They adopted the Gallo-Romance language of the Frankish land they settled, their dialect becoming known as Norman, Normaund or Norman French, an important literary language. The Duchy of Normandy, which they formed by treaty with the French crown, was a great fief of medieval France, and under Richard I of Normandy was forged into a cohesive and formidable principality in feudal tenure. The Normans are noted both for their culture, such as their unique Romanesque architecture and musical traditions, and for their significant military accomplishments and innovations. Norman adventurers founded the Kingdom of Sicily under Roger II after conquering southern Italy on the Saracens and Byzantines, and an expedition o

## Ranking Documents
A rank for a document can be calculated using the **sum** or the **product** of the tf-idf scores of each word in the query calculated over the documents

* Why weight the words by tf-idf?
* What difference does using the sum or the product make intuitively?

Alternatively, you can calculate the cosine similarity of the query to each document.

* What information does using cosine incorporate that is ignored by the simple sum or product measures?


### Exercise 2.1
Generate tf-idf representations of each of the paragraphs (look back to earlier labs to review how to do this).  Store these representations in the `docsearch` class as well.  Then use one of the ranking schemes described above to rank the documents returned by your `listsearch` method.

Using a product of tf-idf scores, you should find that the first ranked document for the search \['political','conquered'\] is para 1031 with a score of 114; whereas using a sum it is para 1031 with a score of 27.7

In [35]:

def doc_freq(doclist):
    df={}
    for doc in doclist:
        for feat in doc.keys():
            df[feat]=df.get(feat,0)+1
            
    return df
    
def idf(doclist):
    N=len(doclist)
    return {feat:math.log(N/v) for feat,v in doc_freq(doclist).items()}

def convert_to_tfidf(docs,idfvalues):
    converted=[{f:v*idfvalues.get(f,0) for f,v in doc.items()} for doc in docs]
    return converted

In [36]:
class docssearch():
    
    def __init__(self,docs):
        self.docs=docs
        self.index=create_index(docs)        
        self.bow=[make_bow(doc) for doc in self.docs]
        self.tfidf=convert_to_tfidf(self.bow,idf(self.bow))
        
    def search(self,keyword):
        return [self.docs[index] for index in self.index.get(keyword,[])]
        
    def listsearch(self,keywordlist):
        keywordlist=stem(filter_stopwords(normalise(keywordlist)))
        if len(keywordlist)>0:
            selected =self.index.get(keywordlist[0],[])
            for keyword in keywordlist[1:]:
                selected=merge(selected,self.index.get(keyword,[]))
                
        scored=[(ind,self.score(ind,keywordlist)) for ind in selected]
        ranked=sorted(scored,key=operator.itemgetter(1),reverse=True)
        print(ranked)
        return [self.docs[index] for (index,_) in ranked]
    
    def score(self,ind,keywordlist):
        tfidfs=self.tfidf[ind]
        thesum=1
        for keyword in keywordlist:
            thesum*=tfidfs.get(keyword,0)
        return thesum

In [37]:
paras=docssearch(textlist)
paras.listsearch(['norman','duke'])

[(1, 128.3922637569877), (16, 96.29419781774077), (33, 64.19613187849384), (8, 48.14709890887038), (14, 16.04903296962346)]


['The Norman dynasty had a major political, cultural and military impact on medieval Europe and even the Near East. The Normans were famed for their martial spirit and eventually for their Christian piety, becoming exponents of the Catholic orthodoxy into which they assimilated. They adopted the Gallo-Romance language of the Frankish land they settled, their dialect becoming known as Norman, Normaund or Norman French, an important literary language. The Duchy of Normandy, which they formed by treaty with the French crown, was a great fief of medieval France, and under Richard I of Normandy was forged into a cohesive and formidable principality in feudal tenure. The Normans are noted both for their culture, such as their unique Romanesque architecture and musical traditions, and for their significant military accomplishments and innovations. Norman adventurers founded the Kingdom of Sicily under Roger II after conquering southern Italy on the Saracens and Byzantines, and an expedition o

In [38]:
paras.listsearch(['political','conquered'])

[(1031, 114.0579151691559), (1029, 91.24633213532474), (1, 11.405791516915592), (1076, 11.405791516915592)]


["Imperialism and colonialism both dictate the political and economic advantage over a land and the indigenous populations they control, yet scholars sometimes find it difficult to illustrate the difference between the two. Although imperialism and colonialism focus on the suppression of an other, if colonialism refers to the process of a country taking physical control of another, imperialism refers to the political and monetary dominance, either formally or informally. Colonialism is seen to be the architect deciding how to start dominating areas and then imperialism can be seen as creating the idea behind conquest cooperating with colonialism. Colonialism is when the imperial nation begins a conquest over an area and then eventually is able to rule over the areas the previous nation had controlled. Colonialism's core meaning is the exploitation of the valuable assets and supplies of the nation that was conquered and the conquering nation then gaining the benefits from the spoils of 

## Extensions

Implement one or more of the following extensions.

### Extension 1

In order to be able to answer natural language queries, it is also necessary to extract keywords from questions.

You can use the `input()` function to take in a query from keyboard.  Use this together with the functionality you have already developed to make a prototype of a system which returns the most relevant paragraph to a given query. 

### Extension 2

Find the sentences within a paragraph which are most relevant to a query.


### Extension 3

Build a classifier to determine whether a given query is answerable on the basis of a given relevant paragraph. 

You will need to use the `qalist` which has positive and negative examples of answerable questions for each paragraph.

An instance is then a paragraph and an answer pair which has been labelled answerable or unanswerable.  You can extract features from both the paragraph and the answer (e.g., bags-of-words) and then make a representation of the instance by applying some operation to the two vectors e.g., addition, subtraction, multiplication or concatenation.